In [10]:
import os
os.environ["OMP_NUM_THREADS"]='16'
import numpy as np
from scipy import linalg
from functools import reduce
np.set_printoptions(precision=2)
import matplotlib.pyplot as plt
import tensornetwork as tn
import time,math,gc,itertools
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigsh
from scipy.sparse.linalg import svds
from tqdm import tqdm
import opt_einsum
import sys
from colorsys import hls_to_rgb
def show(z):
    if len(z.shape)==1:
        z=z.reshape((1,z.shape[0]))
    r,arg = np.abs(z),np.angle(z)
    h,s = (arg + np.pi)  / (2 * np.pi) + 0.5,0.8
    l=r**(1/2.2) #gamma correction
    l=l/(2*np.max(l))
    c = np.array(np.vectorize(hls_to_rgb) (h,l,s)).swapaxes(0,2).swapaxes(0,1)
    plt.imshow(c)
    plt.show()
MEMORY_LIMIT=30*1024*1024*1024

In [11]:
import threading,time,datetime
import io,sys
from tqdm import tqdm
from IPython.display import clear_output

class BackgroundRunning:
    def __init__(self, job):
        self.result=None
        self.display=io.StringIO()
        self.job=job
    def start(self):
        self.start_time=time.time()
        def run_thread():
            self.result=job(self.display)
        self.thread=threading.Thread(target=run_thread,args=())
        self.thread.start()
                
    def showStatus(self):
        def sec2str(sec):
            return str(datetime.timedelta(seconds=int(sec)))
        
        clear_output(wait=True)
        elapsed_time=time.time()-self.start_time
        s='[{}]'.format(sec2str(elapsed_time))
        print(s)
        s=self.display.getvalue()
        x=s.rfind('\r')
        if x!=-1:
            s=s[x+1:]
        print(s)
        if self.is_done():
            print('done')
    def monitor(self,duration=10):
        for i in range(duration*4):
            self.showStatus()
            if self.is_done():
                return
            time.sleep(.25)
        print('to be continue...')
    def is_done(self):
        return not self.thread.is_alive()

In [12]:
def getCG(n):
    """shape:[n+1]+[2]*n
    """
    if n==0:
        return np.eye(1)
    CG=np.zeros((n+1,)+(2,)*n)
    for i in range(2**n):
        indices=tuple(map(int,bin(i)[2:].zfill(n)))
        m=np.sum(indices)
        CG[(m,)+indices]=1
    CG=np.array(list(map(lambda x:x/linalg.norm(x),CG.reshape(n+1,-1)))).reshape(CG.shape)
    return CG

assert getCG(3).shape==(4,2,2,2)
CGs=[getCG(i) for i in range(7)]
singlet = np.array([[0,1],[-1,0]])
assert (singlet.dot(singlet)==-np.eye(2)).all()
assert (singlet==-singlet.T).all()

In [13]:
def contract_physical_edges(physical_edges1,physical_edges2):
    """
    Parameters
    ----------
    edges1,edges2:map vertice_id->edge
    """
    for k in physical_edges1:
        if k in physical_edges2:
            physical_edges1[k]^physical_edges2[k]
def contract_disconnected_tensor_network(nodes,edges):
    newNodes=[]
    nodes=set(nodes)
    while len(nodes)>0:
        newSet=set(tn.reachable(next(iter(nodes))))
        newEdges=list(filter(lambda x:x.get_nodes()[0] in newSet or x.get_nodes()[1] in newSet,edges))
        #print(newSet,[[x.get_nodes()[0],x.get_nodes()[1]] for x in newEdges])
        newNode=tn.contractors.auto(newSet,newEdges)
        nodes=nodes.difference(newSet)
        newNodes.append(newNode)
    result=tn.outer_product_final_nodes(newNodes,edges)
    return result 

In [14]:
assert sys.version_info>=(3,7) #will use dict ordering feature
class AKLT_Tensor:
    """
    Attributes
    ----------
    physical_edges:sorted dict vertice_id->edge
    virtual_edges:sorted dict vertice_id->edge
    nodes:sorted dict vertice_id->node or None after SVD
    physical_dimension:int
    virtual_dimension:int
    
    """
    def __init__(self,spins,conns,div):
        """
        Parameters
        ----------
        spins:[2,2,2,2]
            the spins for the vertices of the parent graph
        conns:[[0,1],[1,2],[2,3]]
            array of pair of vertice_ids for edges in the parent graph
        div:[1,2]
            the set of vertice_ids for vertices in the subgraph
        """        
        self.nodes=dict()
        self.physical_edges=dict()
        self.virtual_edges=dict()
        
        curIndice=[1]*len(spins)
        for vid in div:
            self.nodes[vid]=tn.Node(CGs[spins[vid]])# shape: [n+1]+[2]*n
        for vid1,vid2 in conns:
            if vid1 in div and vid2 in div:
                n1=self.nodes[vid1]
                n2=tn.Node(singlet)
                n3=self.nodes[vid2]

                edges=n1.get_all_edges()
                edges[curIndice[vid1]]=n2[1]
                n1=tn.contract(n1[curIndice[vid1]]^n2[0]).reorder_edges(edges)
                self.nodes[vid1]=n1

                n1[curIndice[vid1]]^n3[curIndice[vid2]]
                curIndice[vid1]+=1
                curIndice[vid2]+=1
        for vid in div:
            n1=self.nodes[vid]
            self.physical_edges[vid]=n1[0]
            num=spins[vid]+1-curIndice[vid]
            if num==0:
                pass
            elif num==1:
                self.virtual_edges[vid]=n1[curIndice[vid]]
            else:
                n2=tn.Node(CGs[num].conj())
                edges=n1.get_all_edges()[0:curIndice[vid]]+[n2[0]]
                for i in range(num):
                    n1[curIndice[vid]+i]^n2[1+i]
                n1=tn.contract_between(n1,n2,output_edge_order=edges)
                self.nodes[vid]=n1
                self.virtual_edges[vid]=edges[-1]
        
        self.nodes=dict(sorted(self.nodes.items()))
        self.physical_edges=dict(sorted(self.physical_edges.items()))
        self.virtual_edges=dict(sorted(self.virtual_edges.items()))
        
        self.physical_dimensions=[e.dimension for e in self.physical_edges.values()]
        self.virtual_dimensions=[e.dimension for e in self.virtual_edges.values()]
        self.physical_dimension=np.prod(self.physical_dimensions)
        self.virtual_dimension=np.prod(self.virtual_dimensions)
        

    
    def print_dimensions(self):
        print("node shapes:",[(i,n.shape) for i,n in self.nodes.items()])
        print("physical dimensions:",self.physical_dimensions)
        print("virtual dimensions:",self.virtual_dimensions)
        print("contracted size:",self.physical_dimension,"x",self.virtual_dimension,"=",self.physical_dimension*self.virtual_dimension)
    
    def print_topology(self):
        inm={n:i for i,n in self.nodes.items()}
        toPrint=[]
        for n in self.nodes.values():
            for e in n.edges:
                if e in self.physical_edges.values():
                    assert e.node2 is None
                    toPrint.append([inm[e.node1],'p',e.dimension])
                elif e in self.virtual_edges.values():
                    assert e.node2 is None
                    toPrint.append([inm[e.node1],'v',e.dimension])
                else:
                    toPrint.append([e.node1 and inm[e.node1],e.node2 and inm[e.node2],e.dimension])
        print(toPrint)
    def copy(self):
        node_dict,edge_dict=tn.copy(self.nodes.values())
        rtval=object.__new__(self.__class__)
        
        rtval.nodes={k:node_dict[v] for k,v in self.nodes.items()}
        rtval.physical_edges={k:edge_dict[v] for k,v in self.physical_edges.items()}
        rtval.virtual_edges={k:edge_dict[v] for k,v in self.virtual_edges.items()}
        rtval.physical_dimensions=self.physical_dimensions.copy()
        rtval.virtual_dimensions=self.virtual_dimensions.copy()
        rtval.physical_dimension=self.physical_dimension
        rtval.virtual_dimension=self.virtual_dimension
        
        return rtval
    def get_tensor(self):
        t=self.copy()
        return contract_disconnected_tensor_network(
            list(t.nodes.values()),
            list(t.physical_edges.values())+list(t.virtual_edges.values())
        ).tensor


In [15]:
class AKLT_Physical_Basis:
    def __init__(self,spins,conns,div,validate=True):
        """
        Parameters
        ----------
        spins:[2,2,2,2]
            the spins for the vertices of the parent graph
        conns:[[0,1],[1,2],[2,3]]
            array of pair of vertice_ids for edges in the parent graph
        div:[1,2]
            the set of vertice_ids for vertices in the subgraph
        """
        tensor=AKLT_Tensor(spins,conns,div)
        # assert tn.check_correct(tensor.nodes.values())==None wrong for non-connected graph

        
        t1,t2=tensor.copy(),tensor.copy()
        contract_physical_edges(t1.physical_edges,t2.physical_edges)
        mm=contract_disconnected_tensor_network(
            list(t1.nodes.values())+list(t2.nodes.values()),
            list(t1.virtual_edges.values())+list(t2.virtual_edges.values())
        ).tensor
        
        print("begin SVD for ",tensor.virtual_dimension," x ",tensor.virtual_dimension)
        s2,v=linalg.eigh(mm.reshape((tensor.virtual_dimension,tensor.virtual_dimension)))
        s=np.sqrt(s2)
        vh=v.conj().T
        assert np.allclose(v@vh,np.eye(v.shape[0]))
        assert np.allclose(vh@v,np.eye(vh.shape[0]))
        print("maxmin singular values: ",max(s),min(s)," total ",len(s))
        
        #new_tensor=v@np.diag(1/s)
        new_tensor=v@np.diag(1/s)@vh
        new_node=tn.Node(new_tensor.reshape(tensor.virtual_dimensions+[new_tensor.shape[1]]))
        
        for i,e in enumerate(tensor.virtual_edges.values()):
            new_node[i]^e
                
        self.all_nodes=[new_node]+list(tensor.nodes.values())
        self.physical_edges=tensor.physical_edges
        self.virtual_edge=new_node[-1]
        self.virtual_dimension=new_tensor.shape[1]
        self.physical_dimensions=tensor.physical_dimensions.copy()
        self.physical_dimension=tensor.physical_dimension
        self.singular_values=s
        self.v_tensor=v
        
        if validate:
            print('start validate')
            self.test_unitary()
            self.test_idempotent()
            print('validate passed')
        
    def print_dimensions(self):
        print("node shapes:",[n.shape for n in self.all_nodes])
        print("physical dimensions:",self.physical_dimensions)
        print("virtual dimension:",self.virtual_dimension)
        print("contracted size:",self.physical_dimension,"x",self.virtual_dimension,"=",self.physical_dimension*self.virtual_dimension)
    
    def copy(self):
        node_dict,edge_dict=tn.copy(self.all_nodes)
        rtval=object.__new__(self.__class__)
        
        rtval.all_nodes=[node_dict[v] for v in self.all_nodes]
        rtval.physical_edges={k:edge_dict[v] for k,v in self.physical_edges.items()}
        rtval.virtual_dimension=self.virtual_dimension
        rtval.virtual_edge=edge_dict[self.virtual_edge]
        rtval.physical_dimensions=self.physical_dimensions.copy()
        rtval.physical_dimension=self.physical_dimension
        rtval.singular_values=self.singular_values.copy()
        rtval.v_tensor=self.v_tensor.copy()
        
        return rtval
    def get_tensor(self):
        t=self.copy()
        return contract_disconnected_tensor_network(
            t.all_nodes,
            list(t.physical_edges.values())+[t.virtual_edge]
        ).tensor
    
    def test_unitary(self):
        a,b=self.copy(),self.copy()
        contract_physical_edges(a.physical_edges,b.physical_edges)
        t=contract_disconnected_tensor_network(
            a.all_nodes+b.all_nodes,
            [a.virtual_edge,b.virtual_edge]
        ).tensor
        assert np.allclose(t,np.eye(self.virtual_dimension))
    def test_idempotent(self):
        def E(v):
            n0=tn.Node(v.reshape(self.physical_dimensions))
            n0_physical_edges={k:n0[i] for i,k in enumerate(self.physical_edges)}
            a,b=self.copy(),self.copy()
            contract_physical_edges(n0_physical_edges,a.physical_edges)
            a.virtual_edge^b.virtual_edge
            nodes=a.all_nodes+b.all_nodes+[n0]
            edges=list(b.physical_edges.values())
            n1=contract_disconnected_tensor_network(nodes,edges)
            return n1.tensor.flatten()
        v=np.random.random(self.physical_dimension)
        assert np.allclose(E(v),E(E(v)))
        
a=AKLT_Physical_Basis([4,2],[[0,1]],[0,1])
b=AKLT_Tensor([4,2],[[0,1]],[0,1])
pd=a.physical_dimension
vd=a.virtual_dimension
ua=a.get_tensor().reshape(pd,-1)@a.v_tensor
sa=np.diag(a.singular_values)
va=a.v_tensor
tb=b.get_tensor().reshape(pd,-1)
assert np.allclose(ua@sa@va.T,tb)
assert np.allclose(va.T@va,np.eye(vd))
assert np.allclose(va@va.T,np.eye(vd))
assert np.allclose(ua.T@ua,np.eye(vd))
assert np.allclose(ua@ua.T,ua@ua.T@ua@ua.T)

a.test_unitary()
a.test_idempotent()


begin SVD for  8  x  8
maxmin singular values:  1.1180339887498951 0.8660254037844386  total  8
start validate
validate passed


In [24]:
# Kagome Decorated 4-Triangle Subgraph

# 5653it [1:34:14,  1.00s/it, 2.0323123973306525>2.0221388836559346]
# eta0  0.03 tol  0 eta  0.061837628687323176

# 21it [00:22,  1.05s/it, 0.6964883420000261>1.938592142278469]
# eta0  0.05 tol  0.01 eta  0.06134625404240779
spins=[4,2,4,2,4,2,4,2,4,2,4,2,2,2,2,4,2,4,2,4,2,4,2,4,2,4,2,4,2,4,2,4,2,2,2,2]
def c(l):
    rtval=[]
    for i in range(len(l)):
        rtval+=[l[i],l[(i+1)%len(l)]]
    return rtval
conns=c([1,2,3,4,5,6])+c([7,8,9,10,11,12])+c([1,14,7,15,16,13])+c([16,17,18,19,20,21])+c([22,23,24,25,26,27])+c([28,29,30,31,32,33])+c([20,34,22,35,28,36])
connsL=np.array(conns).reshape(-1,2)-1
connsM=connsL
connsR=connsL
connsLM=connsL
connsMR=connsL
left=range(0,15)
middle=range(15,21)
right=range(21,36)
check_dimL=2**2*3**4
check_dimM=3**3
check_dimR=2**2*3**4

In [25]:
memory_limit=30*1024*1024*1024
left,middle,right=set(left),set(middle),set(right)
assert all(all(i<j for i in left)for j in middle)
assert all(all(i<j for i in middle)for j in right)
assert len(left)+len(middle)+len(right)==max(right)+1

UL=AKLT_Physical_Basis(spins,connsL,left,validate=False)
UM=AKLT_Physical_Basis(spins,connsM,middle,validate=False)
UR=AKLT_Physical_Basis(spins,connsR,right,validate=False)
ULM=AKLT_Physical_Basis(spins,connsLM,left.union(middle),validate=False)
UMR=AKLT_Physical_Basis(spins,connsMR,middle.union(right),validate=False)

dimL,dimM,dimR=UL.virtual_dimension,UM.virtual_dimension,UR.virtual_dimension
dimLMR=dimL*dimM*dimR


def E(v):
    n0=tn.Node(v.reshape((UL.virtual_dimension,UM.virtual_dimension,UR.virtual_dimension)))
    n1,n2=UL.copy(),UM.copy()
    n3=ULM.copy()
    n4=ULM.copy()
    n5,n6=UL.copy(),UM.copy()
    n0[0]^n1.virtual_edge
    n0[1]^n2.virtual_edge
    contract_physical_edges(n1.physical_edges,n3.physical_edges)
    contract_physical_edges(n2.physical_edges,n3.physical_edges)
    n3.virtual_edge^n4.virtual_edge
    contract_physical_edges(n5.physical_edges,n4.physical_edges)
    contract_physical_edges(n6.physical_edges,n4.physical_edges)
    
    nodes=[n0]+[n for N in [n1,n2,n3,n4,n5,n6] for n in N.all_nodes]
    edges=[n5.virtual_edge,n6.virtual_edge,n0[2]]
    
    n7=tn.contractors.auto(nodes,edges,memory_limit=memory_limit)
    rtval=n7.tensor.reshape(-1)
    return rtval

def F(v):
    n0=tn.Node(v.reshape((UL.virtual_dimension,UM.virtual_dimension,UR.virtual_dimension)))
    n1,n2=UM.copy(),UR.copy()
    n3=UMR.copy()
    n4=UMR.copy()
    n5,n6=UM.copy(),UR.copy()
    n0[1]^n1.virtual_edge
    n0[2]^n2.virtual_edge
    contract_physical_edges(n1.physical_edges,n3.physical_edges)
    contract_physical_edges(n2.physical_edges,n3.physical_edges)
    n3.virtual_edge^n4.virtual_edge
    contract_physical_edges(n5.physical_edges,n4.physical_edges)
    contract_physical_edges(n6.physical_edges,n4.physical_edges)
    
    nodes=[n0]+[n for N in [n1,n2,n3,n4,n5,n6] for n in N.all_nodes]
    edges=[n0[0],n5.virtual_edge,n6.virtual_edge]
    
    n7=tn.contractors.auto(nodes,edges,memory_limit=memory_limit)
    rtval=n7.tensor.reshape(-1)
    return rtval

assert dimL==check_dimL and dimM==check_dimM and dimR==check_dimR
print('dimL*dimM*dimR',dimL*dimM*dimR)
print("--- start timing ---")
start_time = time.time()
v=np.random.random(dimLMR)
Ev=E(v)
Fv=F(v)
assert np.allclose(E(Ev),Ev)
assert np.allclose(F(Fv),Fv)
del v
print("--- %s seconds ---" % (time.time() - start_time))
print('done')

begin SVD for  324  x  324
maxmin singular values:  4.43705464600758 3.1004785661183063  total  324
begin SVD for  27  x  27
maxmin singular values:  1.5858897490986876 1.2123973801552888  total  27
begin SVD for  324  x  324
maxmin singular values:  4.4370546460075815 3.1004785661183054  total  324
begin SVD for  729  x  729
maxmin singular values:  10.68053159407836 6.85149275471752  total  729
begin SVD for  729  x  729
maxmin singular values:  10.680531594078362 6.851492754717523  total  729
dimL*dimM*dimR 2834352
--- start timing ---
--- 0.9566662311553955 seconds ---
done


In [26]:
import psutil,os
print('memory usage',psutil.Process(os.getpid()).memory_info().rss)

memory usage 1249120256


In [27]:
del bg
gc.collect(2)

14469

In [28]:
eta=0.03
tol=0
threshold=2*(1+eta)




def job(file):
    pbar=tqdm(file=file)
    oldeigs=[0]
    def OP(v):
        Ev=E(v)
        gc.collect(2)
        Fv=F(v)
        gc.collect(2)
        EFv=E(Fv)
        gc.collect(2)
        FEv=F(Ev)
        gc.collect(2)
        rtval=(2+eta)*(Ev+Fv)-(EFv+FEv)
        pbar.update(1)
        del Ev,Fv,EFv,FEv
        gc.collect(2)
        eig=np.linalg.norm(rtval)/np.linalg.norm(v)
        pbar.set_postfix_str(str(oldeigs[-1])+'>'+str(eig))
        oldeigs.append(eig)
        return rtval
    op=LinearOperator(shape=(dimLMR,dimLMR),matvec=OP)
    eigenvalues, eigenvectors = eigsh(op, k=1,tol=tol)
    pbar.close()
    return eigenvalues,eigenvectors

bg=BackgroundRunning(job)
bg.start()



/home/wenhan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars


In [33]:
bg.monitor(100)
if(bg.is_done()):
    eigenvalues, eigenvectors=bg.result

    e=eigenvalues[0]
    print(e,'<',threshold,'==',e<threshold)
    print("possible -a, a should be bigger than eta:")
    print(0.5*(-1-eta+ np.sqrt(9 - 4 *e + 6 *eta + eta**2)))
    print(0.5*(-1-eta- np.sqrt(9 - 4 *e + 6 *eta + eta**2)))

    smallRoot=-0.5*(-1-eta+ np.sqrt(9 - 4 *e + 6 *eta + eta**2))
    print("eta0 ",eta,"tol ",tol,"eta ",smallRoot)

[1:56:14]
5653it [1:34:14,  1.00s/it, 2.0323123973306525>2.0221388836559346]

done
2.0898688652262716 < 2.06 == False
possible -a, a should be bigger than eta:
-0.061837628687323176
-0.9681623713126768
eta0  0.03 tol  0 eta  0.061837628687323176


In [ ]:
bg.thread.raise_exception() 

In [ ]:
1+1